In [0]:
# este script tenta reproducir os resultados do artigo "BERT for Stock Market Sentiment Analysis" do 
# autoria deles.

# autor deste codigo: steve

# notas: verificar que o Runtime do Colab seja GPU e Python 3.

import datetime
import json
import os
import pprint
import random
import string
import sys
import tensorflow as tf


# clonamos o git
!test -d bert_repo || git clone https://github.com/stonescenter/bert.git bert_repo
if not 'bert_repo' in sys.path:
  sys.path += ['bert_repo']

if 'COLAB_TPU_ADDR' not in os.environ: 
  print('Not connected to TPU') 
else: 
  print("Connected to TPU, please use GPU")
  

Cloning into 'bert_repo'...
remote: Enumerating objects: 468, done.
remote: Total 468 (delta 0), reused 0 (delta 0), pack-reused 468
Receiving objects: 100% (468/468), 633.95 KiB | 848.00 KiB/s, done.
Resolving deltas: 100% (270/270), done.
Not connected to TPU


In [0]:
# Available pretrained model checkpoints:
#   uncased_L-12_H-768_A-12: uncased BERT base model
#   uncased_L-24_H-1024_A-16: uncased BERT large model
#   cased_L-12_H-768_A-12: cased BERT large model
BERT_MODEL = 'uncased_L-12_H-768_A-12'
BERT_BASE_DIR='gs://bert_models/2018_10_18/uncased_L-12_H-768_A-12'

BERT_MODEL_HUB = 'https://tfhub.dev/google/bert_' + BERT_MODEL + '/1'  
#BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"


# configuramos o modelo pre-treinado do bert
BERT_PRETRAINED_DIR = 'gs://cloud-tpu-checkpoints/bert/' + BERT_MODEL 
print('***** BERT pretrained directory: {} *****'.format(BERT_PRETRAINED_DIR))

!gsutil ls $BERT_PRETRAINED_DIR
BERT_BASE_DIR = BERT_PRETRAINED_DIR
!gsutil ls $BERT_BASE_DIR

  

***** BERT pretrained directory: gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12 *****
gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12/bert_config.json
gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001
gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12/bert_model.ckpt.index
gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12/bert_model.ckpt.meta
gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12/checkpoint
gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12/vocab.txt
gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12/bert_config.json
gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001
gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12/bert_model.ckpt.index
gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12/bert_model.ckpt.meta
gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12/checkpoint
gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12/vocab.txt

In [0]:
# dados temporales durante a session em colab
DATA_TRAIN ='bert_repo/BNEWS_DATA/datasetEconomyNews_PN.json'
DATA_TEST = 'bert_repo/BNEWS_DATA/predict_data.json'

TASK_NAME = 'bnd'

# bucket propio para almacenar se nao tem entao sera eliminado constantemente
# em cada sesion. Esta opcao precisa pagar no google para storage e login
#BUCKET = 'hd-storage-bucket'
#OUTPUT_DIR = 'gs://{}/bert/models/{}'.format(BUCKET, TASK_NAME)

OUTPUT_DIR = 'output/bert/{}'.format(TASK_NAME)
tf.gfile.MakeDirs(OUTPUT_DIR)
print('***** Directorio creado : {} *****'.format(OUTPUT_DIR))

#!gsutil ls -la $OUTPUT_DIR
!test -d $OUTPUT_DIR
if not OUTPUT_DIR in sys.path:
  sys.path += [OUTPUT_DIR]
  
#with tf.gfile.GFile(OUTPUT_DIR + '/dataDistribution.txt', 'a') as f:
#  f.write("------ STATISTICS %s ------\n")
#f.close()
#!gsutil cat $DIST

#comando para editar un file
#%pycat bert_repo/run_classifier.py
#!rm bert_repo/run_classifier.py
#%%writefile bert_repo/run_classifier.py



***** Directorio creado : output/bert/bnd *****


In [0]:
# verificamos se funciona a variavel DATA_TEST dados do teste
data_predict = []
with tf.gfile.Open(DATA_TEST,'r') as fp:
  data_predict = json.load(fp)

for block in data_predict:
  title, text =  block["headlineTitle"], block["headlineText"]
  print(text)
print(DATA_TEST)

In [0]:
# mandamos a treinar os dados.
!python bert_repo/run_classifier.py \
--task_name=$TASK_NAME \
--do_train=true \
--do_eval=true \
--data_dir=$DATA_TRAIN \
--vocab_file=$BERT_BASE_DIR/vocab.txt \
--bert_config_file=$BERT_BASE_DIR/bert_config.json \
--init_checkpoint=$BERT_BASE_DIR/bert_model.ckpt \
--max_seq_length=64 \
--train_batch_size=32 \
--learning_rate=2e-5 \
--num_train_epochs=10.0 \
--output_dir=$OUTPUT_DIR \
--seed=123124124




W1003 14:43:59.926418 139824250898304 deprecation_wrapper.py:119] From bert_repo/run_classifier.py:911: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.


W1003 14:43:59.926634 139824250898304 deprecation_wrapper.py:119] From bert_repo/run_classifier.py:911: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.


W1003 14:43:59.926843 139824250898304 deprecation_wrapper.py:119] From bert_repo/run_classifier.py:325: The name tf.gfile.Open is deprecated. Please use tf.io.gfile.GFile instead.

Erro ao tentar abrir o diretório[Errno 17] File exists: 'output/bert/bnd'

W1003 14:43:59.930906 139824250898304 deprecation_wrapper.py:119] From bert_repo/run_classifier.py:359: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

Erro ao tentar abrir o diretório[Errno 17] File exists: 'output/bert/bnd'
Erro ao tentar abrir o diretório[Errno 17] File exists: 'output/bert/bnd'
2019-10-0

In [0]:
# mandamos a testar os dados.  Aqui sai um error de incompatibilidade, o motivo parecer ser o tensorflow e os checkpoint
# e nao ao codigo.
!python bert_repo/run_classifier.py \
--task_name=$TASK_NAME \
--do_predict=true \
--data_dir=$DATA_TEST \
--vocab_file=$BERT_BASE_DIR/vocab.txt \
--bert_config_file=$BERT_BASE_DIR/bert_config.json \
--init_checkpoint=$BERT_BASE_DIR/bert_model.ckpt \
--max_seq_length=64 \
--output_dir=$OUTPUT_DIR